<a href="https://colab.research.google.com/github/yashtiwarii0/Python_summarizer/blob/main/summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install openai jsonschema --quiet
!pip install python-dotenv


In [19]:


# End-to-End: Conversation Management & JSON Extraction (Groq API)



# Step 1: Load Groq API Key

from getpass import getpass
import os
from openai import OpenAI



GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    GROQ_API_KEY = getpass("⚠️ Please enter your GROQ API key to run this notebook: ")


client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

print("✅ Groq API key loaded successfully!")


# Step 2: Conversation Manager

class ConversationManager:
    def __init__(self, summary_model="llama-3.3-70b-versatile", summarize_every_k_runs=3, keep_recent_turns_after_summary=2):
        self.history = []
        self.run_count = 0
        self.summary_model = summary_model
        self.summarize_every_k_runs = summarize_every_k_runs
        self.keep_recent_turns_after_summary = keep_recent_turns_after_summary
        self.client = client

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})
        self.run_count += 1
        if self.run_count % self.summarize_every_k_runs == 0:
            self.summarize_history()

    def summarize_history(self):
        try:
            response = self.client.chat.completions.create(
                model=self.summary_model,
                messages=[
                    {"role": "system", "content": "Summarize the following conversation briefly."},
                    {"role": "user", "content": str(self.history)}
                ],
                max_tokens=150
            )
            summary = response.choices[0].message.content.strip()
            self.history = [{"role": "system", "content": f"Conversation summary: {summary}"}] + \
                           self.history[-self.keep_recent_turns_after_summary:]
        except Exception as e:
            print("Summarization API error:", e)

    def get_history(self):
        return self.history

    def pretty_print_history(self):
        print("="*50)
        for i, msg in enumerate(self.history, 1):
            print(f"{i}. [{msg['role'].upper()}] {msg['content']}")
        print("="*50)

    # Truncation helpers
    def truncate_by_turns(self, n):
        return self.history[-n:]

    def truncate_by_chars(self, max_chars):
        result, total = [], 0
        for msg in reversed(self.history):
            if total + len(msg["content"]) <= max_chars:
                result.insert(0, msg)
                total += len(msg["content"])
            else:
                break
        return result

    def truncate_by_words(self, max_words):
        result, total = [], 0
        for msg in reversed(self.history):
            count = len(msg["content"].split())
            if total + count <= max_words:
                result.insert(0, msg)
                total += count
            else:
                break
        return result


# Step 3: JSON Extraction Schema

import json

extraction_schema = {
    "name": "extract_user_info",
    "description": "Extract structured user details from chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": []
    }
}

def extract_info_from_chat(chat_text):
    try:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "You are an information extraction assistant. Fill missing fields with empty string or 0 for age."},
                {"role": "user", "content": chat_text}
            ],
            functions=[extraction_schema],
            function_call={"name": "extract_user_info"}
        )
        func_call = response.choices[0].message.function_call
        arguments = func_call.arguments
        defaults = {"name": "", "email": "", "phone": "", "location": "", "age": 0}
        parsed = {**defaults, **json.loads(arguments)}
        try:
            parsed["age"] = int(parsed["age"])
        except:
            parsed["age"] = 0
        return parsed
    except Exception as e:
        print("Error extracting info:", e)
        return None

def validate_schema(data):
    required_keys = ["name", "email", "phone", "location", "age"]
    for key in required_keys:
        if key not in data:
            return False, f"Missing key: {key}"
    if not isinstance(data["age"], int):
        return False, "Age must be integer"
    return True, "Valid schema"


# Step 4: Demo Conversation + JSON Extraction

def demo_combined():
    cm = ConversationManager(summary_model="llama-3.3-70b-versatile", summarize_every_k_runs=3)

    chats = [
        "Hi, my name is Rahul Sharma. I am 25 years old, living in Pune. My email is rahul.sharma@gmail.com and my phone number is 9876543210.",
        "Hello, I'm Ananya, 30 years old from Bangalore. You can reach me at ananya123@yahoo.com or call me on 9123456789.",
        "Good evening, this is John Doe, 22 years old. I live in Mumbai. My contact email is john.doe@outlook.com and phone number is 9988776655."
    ]

    for i, text in enumerate(chats, 1):
        print(f"\n=== Conversation Turn {i} ===")
        cm.add_message("user", text)
        cm.pretty_print_history()

        info = extract_info_from_chat(text)
        if info:
            print("Extracted JSON:", info)
            valid, msg = validate_schema(info)
            print("Validation:", msg)
        else:
            print("Extraction failed.")

# Run demo
demo_combined()


✅ Groq API key loaded successfully!

=== Conversation Turn 1 ===
1. [USER] Hi, my name is Rahul Sharma. I am 25 years old, living in Pune. My email is rahul.sharma@gmail.com and my phone number is 9876543210.
Extracted JSON: {'name': 'Rahul Sharma', 'email': 'rahul.sharma@gmail.com', 'phone': '9876543210', 'location': 'Pune', 'age': 25}
Validation: Valid schema

=== Conversation Turn 2 ===
1. [USER] Hi, my name is Rahul Sharma. I am 25 years old, living in Pune. My email is rahul.sharma@gmail.com and my phone number is 9876543210.
2. [USER] Hello, I'm Ananya, 30 years old from Bangalore. You can reach me at ananya123@yahoo.com or call me on 9123456789.
Extracted JSON: {'name': 'Ananya', 'email': 'ananya123@yahoo.com', 'phone': '9123456789', 'location': 'Bangalore', 'age': 30}
Validation: Valid schema

=== Conversation Turn 3 ===
1. [SYSTEM] Conversation summary: Three individuals, Rahul Sharma, Ananya, and John Doe, introduced themselves, sharing their ages, locations, and contact info